In [ ]:
import pandas as pd
from math import comb
from ax.service.ax_client import AxClient
import time
import sys

sys.path.append('../../../0_helper_functions')
from sdlnano import BayesianOptimizer_Auto
from calculation import BatchCalculation

In [ ]:
# Design space size calculation


def calculate_combinations(num_variables, total_sum, step):
    steps = int(total_sum / step)
    return comb(steps + num_variables - 1, num_variables - 1)

# Define parameters
step = 0.05
total_sum = 1

# Calculate combinations for each group
count_group1 = calculate_combinations(10, total_sum, step)  # 3 Solid lipids + 3 Polymers + 3 Liquid lipids + 1 Drug
count_group2 = calculate_combinations(4, total_sum, step)   # 3 Surfactants + 1 Water

# Calculate total combinations
total_combinations = count_group1 * count_group2


print("organic design space size: ", round(count_group1/(1e6),2), 'Million')
print("aqueous design space size: ", round(count_group2/(1e3),2), 'Thousand')
print("design space size: ", round(total_combinations/(1e9),2), 'Billion')


organic design space size:  10.02 Million
aqueous design space size:  1.77 Thousand
design space size:  17.74 Billion


# Check the iteration number

In [3]:
iteration = BayesianOptimizer_Auto.get_iteration_number()
print("This is the iteration: #", iteration)

This is the iteration: # 9


# Check the drug

In [4]:
drug = 'ACE'
print("This is the drug:", drug)

This is the drug: ACE


# Optimizer initialization (N/A)

# Generate recommendations

In [5]:
time_start = time.time()

previous_optimizer_file_name = '../iteration_' + str(iteration-1) + '/optimizer/optimizer_load_' + str(iteration-1) + '.json'
# generate recomms
ax_client_init = AxClient.load_from_json_file(previous_optimizer_file_name)
unlabeled_trial, ax_client_design = BayesianOptimizer_Auto.generate_trials(ax_client=ax_client_init, num_of_trials = 16, drug = drug, bopt=1)

time_end = time.time()

print(round((time_end-time_start)/60), 'min')

[INFO 02-12 16:36:11] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[INFO 02-12 16:37:27] ax.service.ax_client: Generated new trial 144 with parameters {'Drug_MW': 0.354, 'Drug_LogP': 0.391, 'Drug_TPSA': 0.067, 'Drug': 1.0, 'SL_1': 0.0, 'SL_2': 0.0, 'SL_3': 0.635946, 'LL_1': 0.810469, 'LL_2': 0.0, 'LL_3': 0.0, 'P_1': 0.064926, 'P_2': 0.0, 'P_3': 1.0, 'S_1': 0.0, 'S_2': 0.647865, 'S_3': 0.0, 'Water': 1.0} using model BoTorch.
/Users/zeqing/opt/anaconda3/envs/sdlnano/lib/python3.10/site-packages/ax/core/data.py:288: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return cls(df=pd.concat(dfs, 

16 min


In [ ]:
# process the trails into ratios

unlabeled_trial_processed = BayesianOptimizer_Auto.process_trails(unlabeled_trial)
unlabeled_trial_processed.describe()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.0,16.00,16.0,16.000000,16.0000,16.0,16.000000,16.000000,16.000000,16.000000,16.000000,16.0,16.000000,16.0,16.000000,16.000000,16.000000
mean,151.500000,354.0,3.91,67.0,0.333937,0.0095,0.0,0.127562,0.124688,0.023125,0.039438,0.030812,0.0,0.311125,0.0,0.382312,0.025562,0.592125
std,4.760952,0.0,0.00,0.0,0.077756,0.0380,0.0,0.122518,0.138182,0.071564,0.085532,0.053448,0.0,0.065903,0.0,0.087353,0.102250,0.084628
min,144.000000,354.0,3.91,67.0,0.232000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.232000,0.0,0.182000,0.000000,0.361000
25%,147.750000,354.0,3.91,67.0,0.270250,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.248750,0.0,0.367000,0.000000,0.594000
50%,151.500000,354.0,3.91,67.0,0.327500,0.0000,0.0,0.135500,0.055000,0.000000,0.000000,0.010000,0.0,0.303000,0.0,0.377000,0.000000,0.621500
75%,155.250000,354.0,3.91,67.0,0.410250,0.0000,0.0,0.213750,0.242750,0.000000,0.000000,0.040000,0.0,0.339000,0.0,0.403000,0.000000,0.632000
max,159.000000,354.0,3.91,67.0,0.457000,0.1520,0.0,0.333000,0.352000,0.277000,0.245000,0.211000,0.0,0.442000,0.0,0.639000,0.409000,0.670000


In [7]:
unlabeled_trial
unlabeled_trial.describe()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000,16.000,16.000,16.0,16.000000,16.0,16.000000,16.000000,16.000000,16.000000,16.000000,16.0,16.000000,16.0,16.000000,16.0000,16.000000
mean,151.500000,0.354,0.391,0.067,1.0,0.022495,0.0,0.427304,0.442673,0.087620,0.131031,0.099503,0.0,0.941787,0.0,0.625088,0.0625,0.972737
std,4.760952,0.000,0.000,0.000,0.0,0.089888,0.0,0.430369,0.489602,0.263148,0.298072,0.189750,0.0,0.095111,0.0,0.123881,0.2500,0.109052
min,144.000000,0.354,0.391,0.067,1.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.701164,0.0,0.446259,0.0000,0.563792
25%,147.750000,0.354,0.391,0.067,1.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.850913,0.0,0.579145,0.0000,1.000000
50%,151.500000,0.354,0.391,0.067,1.0,0.000000,0.0,0.307326,0.136148,0.000000,0.000000,0.032101,0.0,1.000000,0.0,0.605160,0.0000,1.000000
75%,155.250000,0.354,0.391,0.067,1.0,0.000000,0.0,0.907934,1.000000,0.000000,0.000000,0.116100,0.0,1.000000,0.0,0.675475,0.0000,1.000000
max,159.000000,0.354,0.391,0.067,1.0,0.359573,0.0,1.000000,1.000000,1.000000,1.000000,0.761912,0.0,1.000000,0.0,1.000000,1.0000,1.000000


In [ ]:
# Convert ratios to volumes

transfer = BatchCalculation.converter(unlabeled_trial_processed)
transfer.describe()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Solvent
count,16.000000,16.0,16.00,16.0,16.000000,16.000,16.0,16.000000,16.000000,16.000000,16.000000,16.000000,16.0,16.000000,16.0,16.000000,16.0000,16.000000,16.000000
mean,151.500000,354.0,3.91,67.0,116.878125,3.325,0.0,44.646875,43.640625,8.093750,13.803125,10.784375,0.0,108.893750,0.0,382.312500,25.5625,592.125000,-0.065625
std,4.760952,0.0,0.00,0.0,27.214451,13.300,0.0,42.881303,48.363823,25.047507,29.936213,18.706823,0.0,23.065991,0.0,87.352709,102.2500,84.627714,0.229288
min,144.000000,354.0,3.91,67.0,81.200000,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,81.200000,0.0,182.000000,0.0000,361.000000,-0.350000
25%,147.750000,354.0,3.91,67.0,94.587500,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,87.062500,0.0,367.000000,0.0000,594.000000,-0.350000
50%,151.500000,354.0,3.91,67.0,114.625000,0.000,0.0,47.425000,19.250000,0.000000,0.000000,3.500000,0.0,106.050000,0.0,377.000000,0.0000,621.500000,0.000000
75%,155.250000,354.0,3.91,67.0,143.587500,0.000,0.0,74.812500,84.962500,0.000000,0.000000,14.000000,0.0,118.650000,0.0,403.000000,0.0000,632.000000,0.000000
max,159.000000,354.0,3.91,67.0,159.950000,53.200,0.0,116.550000,123.200000,96.950000,85.750000,73.850000,0.0,154.700000,0.0,639.000000,409.0000,670.000000,0.350000


In [9]:
#dead_volume = 3.0 # mL

round(transfer[['Drug','SL_1', 'SL_2', 'SL_3', 'LL_1', 'LL_2', 'LL_3', 'P_1', 'P_2', 'P_3', 'S_1', 'S_2', 'S_3', 'Water', 'Solvent']].sum()/1000,1)

Drug       1.9
SL_1       0.1
SL_2       0.0
SL_3       0.7
LL_1       0.7
LL_2       0.1
LL_3       0.2
P_1        0.2
P_2        0.0
P_3        1.7
S_1        0.0
S_2        6.1
S_3        0.4
Water      9.5
Solvent   -0.0
dtype: float64

In [10]:
# file path
ax_client_design_path = "optimizer/optimizer_design_" + str(iteration) + ".json"

unlabeled_trial_path = "data/unlabeled_" + str(iteration) + ".xlsx"

unlabeled_trial_processed_path = "data/unlabeled_processed_" + str(iteration) + ".xlsx"

transfer_path = 'data/transfer_' + str(iteration) + '.xlsx'


In [11]:
ax_client_design.save_to_json_file(ax_client_design_path)

unlabeled_trial.to_excel(unlabeled_trial_path, index=False)

unlabeled_trial_processed.to_excel(unlabeled_trial_processed_path, index=False)

transfer.drop(['Drug_MW', 'Drug_LogP', 'Drug_TPSA'], axis=1).to_excel(transfer_path, index = False)

[INFO 02-12 16:52:46] ax.service.ax_client: Saved JSON-serialized state of optimization to `optimizer/optimizer_design_9.json`.


# Generate new protocol

In [12]:
# Paths
template_protocol = '../../templates/OT_protocol_template.py'

output_script_path = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_.py'


In [13]:
# Update the script with Excel data
updated_script = BatchCalculation.update_transfer_script(template_protocol, transfer_path, output_script_path) 


In [14]:
updated_script

'protocol/iteration_9_OT_2_protocol_.py'

# Perform experiment

# Results

In [15]:
results = BatchCalculation.parameters_and_complexity(iteration)
results

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Complexity,Complexity_STD
0,144,354,3.91,67,0.285,0.000,0,0.181,0.231,0.000,0.000,0.018,0,0.285,0,0.393,0.000,0.607,5,0
1,145,354,3.91,67,0.442,0.000,0,0.056,0.000,0.000,0.000,0.060,0,0.442,0,0.368,0.000,0.632,4,0
2,146,354,3.91,67,0.423,0.152,0,0.000,0.000,0.000,0.000,0.002,0,0.423,0,0.364,0.000,0.636,4,0
3,147,354,3.91,67,0.309,0.000,0,0.196,0.000,0.000,0.186,0.000,0,0.309,0,0.406,0.000,0.594,4,0
4,148,354,3.91,67,0.435,0.000,0,0.182,0.000,0.000,0.000,0.026,0,0.357,0,0.368,0.000,0.632,4,0
5,149,354,3.91,67,0.333,0.000,0,0.000,0.333,0.000,0.000,0.000,0,0.333,0,0.402,0.000,0.598,3,0
6,150,354,3.91,67,0.406,0.000,0,0.000,0.110,0.000,0.200,0.000,0,0.284,0,0.340,0.000,0.660,4,0
7,151,354,3.91,67,0.250,0.000,0,0.250,0.250,0.000,0.000,0.000,0,0.250,0,0.406,0.000,0.594,4,0
8,152,354,3.91,67,0.245,0.000,0,0.000,0.245,0.000,0.245,0.021,0,0.245,0,0.379,0.000,0.621,5,0
9,153,354,3.91,67,0.352,0.000,0,0.000,0.352,0.000,0.000,0.000,0,0.297,0,0.378,0.000,0.622,3,0


In [16]:
size_raw = BatchCalculation.size_raw(iteration)
size_raw

,Data Quality,Item,Size,PD Index
0,Good,A1,259.520631,0.209988
1,Good,A2,247.871126,0.302245
2,Good,A3,246.690203,0.563891
3,Caution,A4,82.154756,0.570796
4,Caution,A5,96.024287,0.570796
5,Caution,A6,92.175678,0.570796
6,Caution,B1,42.043017,0.570796
7,Caution,B2,46.121435,0.570796
8,Caution,B3,43.435954,0.570796
9,Good,B4,277.399072,0.442029


In [17]:
size_processed = BatchCalculation.process_formulations(size_raw)
size_processed

,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,Formulation 1,251.360653,7.091373,0.358708,0.183584,1
1,Formulation 2,1000.000000,0.000000,1.000000,0.000000,0
2,Formulation 3,1000.000000,0.000000,1.000000,0.000000,0
3,Formulation 4,1000.000000,0.000000,1.000000,0.000000,0
4,Formulation 5,233.141397,2.024874,0.209453,0.067954,1
5,Formulation 6,1000.000000,0.000000,1.000000,0.000000,0
6,Formulation 7,170.595009,5.155941,0.172727,0.006630,1
7,Formulation 8,1000.000000,0.000000,1.000000,0.000000,0
8,Formulation 9,183.889240,6.281816,0.175096,0.040824,1
9,Formulation 10,1000.000000,0.000000,1.000000,0.000000,0


In [18]:
results_final = pd.concat([results, size_processed], axis=1)
results_final

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,...,S_3,Water,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,144,354,3.91,67,0.285,0.000,0,0.181,0.231,0.000,...,0.000,0.607,5,0,Formulation 1,251.360653,7.091373,0.358708,0.183584,1
1,145,354,3.91,67,0.442,0.000,0,0.056,0.000,0.000,...,0.000,0.632,4,0,Formulation 2,1000.000000,0.000000,1.000000,0.000000,0
2,146,354,3.91,67,0.423,0.152,0,0.000,0.000,0.000,...,0.000,0.636,4,0,Formulation 3,1000.000000,0.000000,1.000000,0.000000,0
3,147,354,3.91,67,0.309,0.000,0,0.196,0.000,0.000,...,0.000,0.594,4,0,Formulation 4,1000.000000,0.000000,1.000000,0.000000,0
4,148,354,3.91,67,0.435,0.000,0,0.182,0.000,0.000,...,0.000,0.632,4,0,Formulation 5,233.141397,2.024874,0.209453,0.067954,1
5,149,354,3.91,67,0.333,0.000,0,0.000,0.333,0.000,...,0.000,0.598,3,0,Formulation 6,1000.000000,0.000000,1.000000,0.000000,0
6,150,354,3.91,67,0.406,0.000,0,0.000,0.110,0.000,...,0.000,0.660,4,0,Formulation 7,170.595009,5.155941,0.172727,0.006630,1
7,151,354,3.91,67,0.250,0.000,0,0.250,0.250,0.000,...,0.000,0.594,4,0,Formulation 8,1000.000000,0.000000,1.000000,0.000000,0
8,152,354,3.91,67,0.245,0.000,0,0.000,0.245,0.000,...,0.000,0.621,5,0,Formulation 9,183.889240,6.281816,0.175096,0.040824,1
9,153,354,3.91,67,0.352,0.000,0,0.000,0.352,0.000,...,0.000,0.622,3,0,Formulation 10,1000.000000,0.000000,1.000000,0.000000,0


In [19]:
# µg/mL
results_final['Solu'] = results_final['Drug'] * 600 / 0.3
results_final['Solu_STD'] = 0

results_final.loc[results_final['Formulation Quality'] == 0, 'Solu'] = 0
results_final

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,144,354,3.91,67,0.285,0.000,0,0.181,0.231,0.000,...,5,0,Formulation 1,251.360653,7.091373,0.358708,0.183584,1,570.0,0
1,145,354,3.91,67,0.442,0.000,0,0.056,0.000,0.000,...,4,0,Formulation 2,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
2,146,354,3.91,67,0.423,0.152,0,0.000,0.000,0.000,...,4,0,Formulation 3,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
3,147,354,3.91,67,0.309,0.000,0,0.196,0.000,0.000,...,4,0,Formulation 4,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
4,148,354,3.91,67,0.435,0.000,0,0.182,0.000,0.000,...,4,0,Formulation 5,233.141397,2.024874,0.209453,0.067954,1,870.0,0
5,149,354,3.91,67,0.333,0.000,0,0.000,0.333,0.000,...,3,0,Formulation 6,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
6,150,354,3.91,67,0.406,0.000,0,0.000,0.110,0.000,...,4,0,Formulation 7,170.595009,5.155941,0.172727,0.006630,1,812.0,0
7,151,354,3.91,67,0.250,0.000,0,0.250,0.250,0.000,...,4,0,Formulation 8,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
8,152,354,3.91,67,0.245,0.000,0,0.000,0.245,0.000,...,5,0,Formulation 9,183.889240,6.281816,0.175096,0.040824,1,490.0,0
9,153,354,3.91,67,0.352,0.000,0,0.000,0.352,0.000,...,3,0,Formulation 10,1000.000000,0.000000,1.000000,0.000000,0,0.0,0


In [20]:
results_normalized = BayesianOptimizer_Auto.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,1,0.251361,0.007091,0.285,0.0,0.358708,0.183584,0.416667
1,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.333333
2,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.333333
3,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.333333
4,1,0.233141,0.002025,0.435,0.0,0.209453,0.067954,0.333333
5,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.250000
6,1,0.170595,0.005156,0.406,0.0,0.172727,0.006630,0.333333
7,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.333333
8,1,0.183889,0.006282,0.245,0.0,0.175096,0.040824,0.416667
9,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.250000


In [21]:
results_normalized = BayesianOptimizer_Auto.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,1,0.251361,0.007091,0.285,0.0,0.358708,0.183584,0.416667
1,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.333333
2,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.333333
3,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.333333
4,1,0.233141,0.002025,0.435,0.0,0.209453,0.067954,0.333333
5,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.250000
6,1,0.170595,0.005156,0.406,0.0,0.172727,0.006630,0.333333
7,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.333333
8,1,0.183889,0.006282,0.245,0.0,0.175096,0.040824,0.416667
9,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.250000


In [22]:
labeled_data_path = 'data/labeled_' + str(iteration) + '.xlsx'
labeled_norm_path = 'data/labeled_norm_' + str(iteration) + '.xlsx'


In [23]:
results_final.to_excel(labeled_data_path, index=False)
results_normalized.to_excel(labeled_norm_path, index=False)

# Load the labeled data to update the optimizer

In [24]:
labeled_norm = pd.read_excel(labeled_norm_path)
labeled_norm.head()

,trial_index,Drug_MW,Drug_LogP,Drug_TPSA,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,144,354,3.91,67,0.285,0.000,0,0.181,0.231,0.0,...,0.416667,0,Formulation 1,0.251361,0.007091,0.358708,0.183584,1,0.285,0
1,145,354,3.91,67,0.442,0.000,0,0.056,0.000,0.0,...,0.333333,0,Formulation 2,1.000000,0.000000,1.000000,0.000000,0,0.000,0
2,146,354,3.91,67,0.423,0.152,0,0.000,0.000,0.0,...,0.333333,0,Formulation 3,1.000000,0.000000,1.000000,0.000000,0,0.000,0
3,147,354,3.91,67,0.309,0.000,0,0.196,0.000,0.0,...,0.333333,0,Formulation 4,1.000000,0.000000,1.000000,0.000000,0,0.000,0
4,148,354,3.91,67,0.435,0.000,0,0.182,0.000,0.0,...,0.333333,0,Formulation 5,0.233141,0.002025,0.209453,0.067954,1,0.435,0


In [25]:
ax_client_design = AxClient.load_from_json_file(ax_client_design_path)


[INFO 02-13 08:29:59] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.


In [26]:
loaded_optimizer_path = "optimizer/optimizer_load_" + str(iteration) + ".json"

In [27]:
loaded_ax_client = BayesianOptimizer_Auto.load_labeled_data(ax_client_design, labeled_norm_path)
loaded_ax_client.save_to_json_file(loaded_optimizer_path)

[INFO 02-13 08:29:59] ax.service.ax_client: Completed trial 144 with data: {'Solu': (0.285, 0), 'Size': (0.251361, 0.007091), 'PDI': (0.358708, 0.183584), 'Complexity': (0.416667, 0)}.
[INFO 02-13 08:29:59] ax.service.ax_client: Completed trial 145 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0), 'Complexity': (0.333333, 0)}.
[INFO 02-13 08:29:59] ax.service.ax_client: Completed trial 146 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0), 'Complexity': (0.333333, 0)}.
[INFO 02-13 08:29:59] ax.service.ax_client: Completed trial 147 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0), 'Complexity': (0.333333, 0)}.
[INFO 02-13 08:29:59] ax.service.ax_client: Completed trial 148 with data: {'Solu': (0.435, 0), 'Size': (0.233141, 0.002025), 'PDI': (0.209453, 0.067954), 'Complexity': (0.333333, 0)}.
[INFO 02-13 08:29:59] ax.service.ax_client: Completed trial 149 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0), 'Complex